In [1]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=708ba353c4e02821e2954d4f5ed209a7c0b4d179c474522fb74eba57e68f7967
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [2]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd

In [4]:
ua = UserAgent()
header = {'user-agent': ua.chrome} 

In [44]:
def scraping_actor_data(page):
  link = 'https://mydramalist.com'
  actor_link = 'https://mydramalist.com/search?adv=people&na=3&so=popular&page='
  actor_resp = requests.get(actor_link+page, headers=header)
  if actor_resp.status_code == 200:
    actor_html = actor_resp.text
    actor_soup = BeautifulSoup(actor_html, 'lxml')
    actor_page = actor_soup.find('div', {'class':'m-t nav-active-border b-primary'})
    box_tag = actor_page.find_all('div', {'class':'box'})
    actor_data = []
    for box in box_tag:
      name_tag = box.find('h6', {'class':'text-primary title'})
      if name_tag is not None:
        actor = name_tag.text.strip()
        name_link = name_tag.find('a').get('href')
        name_page = requests.get(link+name_link, headers=header)
        
        if name_page.status_code == 200:
          name_html = name_page.text
          name_soup =  BeautifulSoup(name_html, 'lxml')
          actor_tag = name_soup.find('ul', {'class':'list m-b-0'})
          detail_actor = actor_tag.find_all('li', {'class':'list-item p-a-0'})
          name_info = [name.text.strip() for name in detail_actor]

          if len(name_info) != 9:
            if not name_info[0].startswith('First Name'):
              name_info.insert(0, 'First Name: N/A')
            if not name_info[1].startswith('Family Name'):
              name_info.insert(1, 'Family: N/A')
            if not name_info[3].startswith('Also Known as'):
              name_info.insert(3, 'Also Known as: N/A')
            
            if not name_info[-1].startswith('Age'):
              name_info.insert(9, 'Age: N/A')
            if not name_info[-2].startswith('Died'):
              name_info.insert(-1, 'Died: N/A')
            if not name_info[-3].startswith('Born'):
              name_info.insert(-2, 'Born: N/A')

          name_info = [info.split(':')[1].strip() if info.split(':')[1].strip() != '' else 'N/A' for info in name_info]
          name_info.insert(0, actor)
          del name_info[1:6]

          cast_data = []
          cast_tag = name_soup.find('div', {'class':'col-lg-8 col-md-8'}).find_all('div', {'class':'box-body'})[1]
          if cast_tag.find('h5', {'class':'header'}).text.strip() == 'Drama':        
            cast_drama = cast_tag.find('table', {'class':'table film-list'})
            for data in cast_drama.find('tbody').find_all('tr'):
              if data is not None:
                year = data.find('td', {'class':'year hidden-sm-down'}).text.strip()
                title = data.find('td', {'class':'title'}).find('a').get('title')
                role = data.find('td', {'class':'role hidden-sm-down'}).find('div', {'class':'text-muted roleid'}).text.strip()
                cast_data.append([year, title, role])
          name_info.append(cast_data)
          actor_data.append(name_info)

  return actor_data

In [48]:
get_actor = []
for p in range(11, 18):
  casting = scraping_actor_data(str(p))
  df = pd.DataFrame(casting)
  df.columns = ['Name','Gender','Born','Died','Age','Drama']
  df = df[~df['Drama'].str.len().eq(0)]
  get_actor.append(df)

df_actor = pd.concat(get_actor).reset_index(drop=True)
df_actor

AttributeError: ignored